Для того, чтобы посчитать смены, следующий алгоритм

Есть соответсвие между операцией, станком, количеству ресурсов

Инфраструктура определяет, сколько одновременно можно сделать операций

И люди тоже привязаны к инфраструктуре

Поэтому надо сделать следующий файл с конфигурацией:

Станок Количество_станков Операция Тип_людей Количество_людей 

Останется только перевести из нормо-часов в дни.

1) Сначала заполняем по максимому дневные смены
2) при нехватке в месяц дневных смен, заполняем ночные 

Как будто это можно сделать средствами пандаса

1. Берём мерджим таблицу с операциями и таблицу конфигурации
2. Нормо-часы делим на количество станков
3. Дублируем каждую строчку со станком в соответсввии с количеством и дропаем количество
4. Получаем остаток от деления количества нормо-часов на 12, записываем это в колонку num_shifts
5. Делаем табличку со списком требуемых сотрудников
6. Прибавляем к num_shifts дату старта, тем самым получая с какого числа по какое будет дневная смена
7. Для тех, кто num_shifts > max_days, прибавляем  к дате старта (num_days - num_shift) (надо учесть, что может быть большой разрыв)

Этот алгоритм верен при параллельных операциях

На деле происходит последовательные операции. Глупо будет думать, что вот они сначала сделают все детали B, чтобы сделать деталь А. Наверное, сначала сделают n деталей В, потом их передадут дальше. По принципу конвейера, это просто даст некотурую начальную задерджку. Возмонжно, она будет в рамках часа, не более. Поэтому такой алгоритм может дать приблизительную оценку.

In [1]:
import pandas as pd

In [2]:
staff_conf = pd.read_excel('../data/StaffConfig.xlsx')
times = pd.read_excel('../data/answ.xlsx').drop(columns=['Unnamed: 0'])

In [3]:
staff_conf

,Станок,Количество станков,Операция,Специалист,Количество людей
0,Лазерный станок 2030,1,Лазерная резка листа,Оператор станок с пу/лазер,1
1,Продольно-фрезерный станок - вертикальный,1,Вертикально-фрезерная,Станочник широкого профиля,5
2,Токарный станок (SLZ-500E),1,Токарная,Станочник широкого профиля,5
3,Пост сборки,1,Сборочная,Слесарь по сборке,2
4,Пост сборки,1,Упаковочная,Слесарь по сборке,1
5,Ленточная пила (MER Shark 281 CCS),1,Ленточно-отрезная,Слесарь по сборке,4
6,Дисковая пила (KKS 45 1NA),1,Слесарная,Слесарь по сборке,1
7,Сварочный пост,1,Сварка полуавтоматом в среде защитного газа (MIG),Эл. Сварщик и п/авт машин,1
8,Листосгибочный станок 7630,1,Листогибочная,Оператор станок с пу/гибка,1
9,Вальцовочный станок (Prinzng RME70/103),1,Вальцовочнуя,Оператор станок с пу/гибка,1


In [4]:
times

,Operation,Time
0,Лазерная резка листа,17.659444
1,Листогибочная,92.652778
2,Окрашивание порошком,17.907833
3,Сборочная,20.750000
4,Сварка полуавтоматом в среде защитного газа (MIG),41.166667
5,Упаковочная,0.833333


In [5]:
merged_times = staff_conf.merge(times, left_on=['Операция'], right_on=['Operation'], how='right')
merged_times

,Станок,Количество станков,Операция,Специалист,Количество людей,Operation,Time
0,Лазерный станок 2030,1,Лазерная резка листа,Оператор станок с пу/лазер,1,Лазерная резка листа,17.659444
1,Листосгибочный станок 7630,1,Листогибочная,Оператор станок с пу/гибка,1,Листогибочная,92.652778
2,Листосгибочный станок 5230,1,Листогибочная,Оператор,1,Листогибочная,92.652778
3,Лента порошковой окраски,1,Окрашивание порошком,Оператор окрасочно-сушильной линии и агрегата,1,Окрашивание порошком,17.907833
4,Лента порошковой окраски,1,Окрашивание порошком,Рабочий по металлопрокату и окраске,1,Окрашивание порошком,17.907833
5,Пост сборки,1,Сборочная,Слесарь по сборке,2,Сборочная,20.750000
6,Сварочный пост,1,Сварка полуавтоматом в среде защитного газа (MIG),Эл. Сварщик и п/авт машин,1,Сварка полуавтоматом в среде защитного газа (MIG),41.166667
7,Пост сборки,1,Упаковочная,Слесарь по сборке,1,Упаковочная,0.833333


In [6]:
unknown_operations = merged_times[merged_times['Станок'].isna()]['Operation'].to_list()
unknown_operations

[]

In [ ]:
merged_times['time_per_machine'] = merged_times['time'] / merged_times['Количество станков']
